# Data Management - Progetto Trenitalia

### 1) Creazione csv contenente tutte le stazioni italiane presenti sul portale viaggiatreno.it

In [59]:
import pandas as pd
import requests
import csv

temp = []
stazioni = []
for i in range(65,91):
    r = requests.get("http://www.viaggiatreno.it/viaggiatrenonew/resteasy/viaggiatreno/autocompletaStazione/" + chr(i))
    temp.append(r.text.split("\n"))
    
for i in temp:
    for j in range(0,len(i)-1):
        stazioni.append(i[j])
    
stripped = (line.strip() for line in lista)
lines = (line.split("|") for line in stripped if line)
with open('stazioni.csv', 'w') as out_file:
    writer = csv.writer(out_file)
    writer.writerow(('nome_stazione', 'id'))
    writer.writerows(lines)


### 2) Creazione di un file json contenente l'identificativo del treno associato al codice della stazione di partenza

In [ ]:
#!/usr/bin/python
import datetime
from threading import Timer
import requests
import pandas as pd
import json
import time
import os

# funzione che crea una directory
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

# funzione che crea il file tratte_treni per il giorno successivo
def crea_tratte_treni():
    
    df = pd.read_csv('stazioni.csv', sep=",")
    stazioni = df["id"]
    mese = "Jan"
    anno = 2019
    giorni_settimana = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
    tratte_treni = {}
    
    giorno = int(time.strftime("%d")) + 1
    tratta = list()
    count = 0
    for i in stazioni:
        count += 1
        for h in range(1,23,2):
            try:
               r = requests.get("http://www.viaggiatreno.it/viaggiatrenonew/resteasy/viaggiatreno/partenze/" 
                                + i + "/" + giorni_settimana[datetime.datetime.today().weekday()] +
                                "%20"+ mese +"%20" + str(giorno) +"%20" + str(anno) + "%20" 
                                + str(h) + ":00:00%20GMT+0100")

            except (requests.ConnectionError, requests.Timeout, requests.RequestException) as e:
                print("OOPS!! Connection Error. Make sure you are connected to Internet.\n")
                print("I'm gonna take a break...\n")
                time.sleep(10)
                r = requests.get("http://www.viaggiatreno.it/viaggiatrenonew/resteasy/viaggiatreno/partenze/" + i + "/" + giorni_settimana[datetime.datetime.today().weekday()] +
                                "%20"+ mese +"%20" + str(giorno) +"%20" + str(anno) + "%20" + str(h) + ":00:00%20GMT+0100")            
            
            except KeyboardInterrupt:
                print("Someone closed the program")
   
            if r.status_code == 200:
                tratta.append(r.json())

        for ora in tratta:
            for treno in ora:
                if treno['codOrigine'] == i:
                    tratte_treni[treno['numeroTreno']] = treno['codOrigine']
        tratta = list()
        print(count)
    
    with open('tratte_treni_' + str(datetime.date.today() + datetime.timedelta(days=1)) +'.json', 'w') as outfile:
        json.dump(tratte_treni, outfile)



### 3) Salvataggio dei dati in locale e in mongodb dei treni in formato json della giornata utilizzando il file tratte_treni creato il giorno precedente

In [ ]:
# funzione che manda le richieste al sito viaggiatreno.it per ogni tratta
#!/usr/bin/python
import datetime
from threading import Timer
import requests
import pandas as pd
import json
import time
import os

import pymongo
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client.dati_viaggiatreno

# timer
x=datetime.datetime.today()
y=x.replace(day=x.day, hour=23, minute=0, second=0, microsecond=0)
delta_t=y-x
secs=delta_t.seconds+1

def richieste_viaggiotreno():
    
    print("Start: " + time.strftime("%H:%M:%S"))
    file = json.loads(open("tratte_treni_" + str(datetime.date.today()) + ".json").read())
    createFolder(time.strftime("%d-%m-%Y"))
    treni = []
    time.strftime("%d-%m-%Y")
    for i in file.keys():
        treni.append(i)
        
    giorno = db[time.strftime("%d-%m-%Y")]
    
    temp = []
    count = 0
    conn_error = 0
    timeout_error = 0
    gen_error = 0
    
    for i in range(0,len(file)):

        try:
            r = requests.get("http://www.viaggiatreno.it/viaggiatrenonew/resteasy/viaggiatreno/andamentoTreno/" 
                             + str(file[treni[i]]) + "/" + str(treni[i]))

        except (requests.ConnectionError, requests.Timeout, requests.RequestException) as e:
            timeout_error += 1
            print("OOPS!! Timeout Error")
            print("I'm gonna take a break...\n")
            time.sleep(10)
            print("I'm restarting from station:",file[treni[i]]," - train: ",treni[i],"\n")
            r = requests.get("http://www.viaggiatreno.it/viaggiatrenonew/resteasy/viaggiatreno/andamentoTreno/" + str(file[treni[i]]) + "/" + str(treni[i]))

        except KeyboardInterrupt:
            print("Someone closed the program")
    
        if r.status_code == 200:
            temp.append(r.json())
            
            with open(time.strftime("%d-%m-%Y") +'/' + str(treni[i]) + '_' + time.strftime("%d-%m-%Y") + '.json', 'w') as outfile:
                json.dump(temp, outfile)
            x = giorno.insert_many(temp)
            
            temp = list()
            print ("File: ", count, " - station: ", file[treni[i]]," - train: ",treni[i],"\n" )
            count += 1

    print("End:" + time.strftime("%H:%M:%S"))
    print("Connection Error: ", conn_error)
    print("Timeout Error: ", timeout_error)
    print("General Error: ", gen_error)

t = Timer(secs, richieste_viaggiotreno)
#t.start()

### Web scraping latitudine e longitudine wikipedia

In [22]:
import re
import requests
from bs4 import BeautifulSoup

nome_stazione = "Stazione_di_Pinerolo"
r = requests.get('https://it.wikipedia.org/wiki/' + nome_stazione)
soup = BeautifulSoup(r.text, "html.parser")
lat = soup.find_all("span", class_="latitude")[0].text
lon = soup.find_all("span", class_="longitude")[0].text

44°53′15″N
7°20′13.2″E


### Geopy

In [ ]:
from geopy.geocoders import Nominatim
import pandas as pd
staz = pd.read_csv("stazioni_mancanti.csv", sep=";")
geolocator = Nominatim(user_agent="university_project6")
count = 0
for i in staz.iterrows():
    string = "station " + str(i[1][1]) + " " + str(i[1][2]) + " " +  str(i[1][3])
    location = geolocator.geocode(string)
    if location != None:
        print(location.raw)
        staz["Lat"][count] = location.latitude
        staz["Lon"][count] = location.longitude
    else:
        print("error!")
    count += 1
staz.to_csv("stazioni_mancanti.csv", sep=";")